In [1]:
import os
os.chdir("/DATA_EDS2/AIGC/2312/xuhr2312/workspace/ControlNet")
os.environ["CUDA_VISIBLE_DEVICES"]="4,5,6"


In [2]:
from datasets.cityscapes import CityscapesDataset

/DATA_EDS2/AIGC/2312/xuhr2312/.conda/envs/controlnet/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
dataset = CityscapesDataset()

In [4]:
dataset.__len__()

3475

In [5]:
full_data=[]
full_label=[]
full_labelIds=[]

with open("datasets/cityscapes.txt", 'r') as f:
    for line in f.readlines():
        data, label = line.strip().split(',')
        full_data.append(data)
        full_label.append(label.replace('labelIds', 'color'))
        full_labelIds.append(label.replace('labelIds', 'labelTrainIds'))

In [6]:
full_data

['/DATA_EDS/gaoha/datasets/cityscapes/leftImg8bit/train/aachen/aachen_000000_000019_leftImg8bit.png',
 '/DATA_EDS/gaoha/datasets/cityscapes/leftImg8bit/train/aachen/aachen_000001_000019_leftImg8bit.png',
 '/DATA_EDS/gaoha/datasets/cityscapes/leftImg8bit/train/aachen/aachen_000002_000019_leftImg8bit.png',
 '/DATA_EDS/gaoha/datasets/cityscapes/leftImg8bit/train/aachen/aachen_000003_000019_leftImg8bit.png',
 '/DATA_EDS/gaoha/datasets/cityscapes/leftImg8bit/train/aachen/aachen_000004_000019_leftImg8bit.png',
 '/DATA_EDS/gaoha/datasets/cityscapes/leftImg8bit/train/aachen/aachen_000005_000019_leftImg8bit.png',
 '/DATA_EDS/gaoha/datasets/cityscapes/leftImg8bit/train/aachen/aachen_000006_000019_leftImg8bit.png',
 '/DATA_EDS/gaoha/datasets/cityscapes/leftImg8bit/train/aachen/aachen_000007_000019_leftImg8bit.png',
 '/DATA_EDS/gaoha/datasets/cityscapes/leftImg8bit/train/aachen/aachen_000008_000019_leftImg8bit.png',
 '/DATA_EDS/gaoha/datasets/cityscapes/leftImg8bit/train/aachen/aachen_000009_00001

In [15]:
len(full_label)

3475

In [18]:
from typing import Union, Tuple, Optional

import matplotlib.pyplot as plt
import torch
from PIL import Image
from diffusers import StableDiffusionPipeline, DDIMInverseScheduler, AutoencoderKL, DDIMScheduler
from torchvision import transforms as tvt


def load_image(imgname: str, target_size: Optional[Union[int, Tuple[int, int]]] = None) -> torch.Tensor:
    pil_img = Image.open(imgname).convert('RGB')
    if target_size is not None:
        if isinstance(target_size, int):
            target_size = (target_size, target_size)
        pil_img = pil_img.resize(target_size, Image.Resampling.LANCZOS)
    return tvt.ToTensor()(pil_img)[None, ...]  # add batch dimension


def img_to_latents(x: torch.Tensor, vae: AutoencoderKL):
    x = 2. * x - 1.
    posterior = vae.encode(x).latent_dist
    latents = posterior.mean * 0.18215
    return latents

device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype = torch.float32

inverse_scheduler = DDIMInverseScheduler.from_pretrained('stabilityai/stable-diffusion-2-1', subfolder='scheduler')
pipe = StableDiffusionPipeline.from_pretrained('stabilityai/stable-diffusion-2-1',
                                                scheduler=inverse_scheduler,
                                                safety_checker=None,
                                                torch_dtype=dtype)
pipe.to(device)
vae = pipe.vae


@torch.no_grad()
def ddim_inversion(imgname: str, num_steps: int = 50, verify: Optional[bool] = False) -> torch.Tensor:

    input_img = load_image(imgname).to(device=device, dtype=dtype)
    latents = img_to_latents(input_img, vae)

    inv_latents, _ = pipe(prompt="", negative_prompt="", guidance_scale=1.,
                          width=input_img.shape[-1], height=input_img.shape[-2],
                          output_type='latent', return_dict=False,
                          num_inference_steps=num_steps, latents=latents)

    # verify
    if verify:
        pipe.scheduler = DDIMScheduler.from_pretrained('stabilityai/stable-diffusion-2-1', subfolder='scheduler')
        image = pipe(prompt="", negative_prompt="", guidance_scale=1.,
                     num_inference_steps=num_steps, latents=inv_latents)
        fig, ax = plt.subplots(1, 2)
        ax[0].imshow(tvt.ToPILImage()(input_img[0]))
        ax[1].imshow(image.images[0])
        plt.savefig("output.png")
        plt.show()
    return inv_latents

The config attributes {'skip_prk_steps': True} were passed to DDIMInverseScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.
Couldn't connect to the Hub: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/stabilityai/stable-diffusion-2-1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f20aa91cbe0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: c1800cc7-63dc-45d0-9414-b756ff93d963)').
Will try to load from local cache.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [19]:
with open('/DATA_EDS2/AIGC/2312/xuhr2312/workspace/ControlNet/datasets/inv.txt', 'w') as f:
    # 按顺序处理每个图像，并将结果存储在文件夹中
    for i, imgname in enumerate(full_data):
        inv = ddim_inversion(imgname, num_steps=20, verify=False)

        # 将结果存储为torch文件
        output_path = f'/DATA_EDS2/AIGC/2312/xuhr2312/workspace/ControlNet/data/cityscapes_inv/inv_{i}.pt'
        torch.save(inv, output_path)

        # 将文件的路径写入txt文件
        f.write(output_path + '\n')

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 